In [1]:
import pandas as pd
from datetime import datetime, date, timedelta
from openpyxl import load_workbook
from openpyxl.styles import Font, Color, Alignment, Border, Side, PatternFill
import shutil
import os, stat
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
oblako_name = '22-10-2024_16-00_Реестр ЦПП.xlsx'
oblako_path = 'z:/Отдел аналитики/2024/10_окт_24/22-10-24/Файлы_для_облака/Для раздачи ВПС/'

oblako = pd.read_excel(oblako_path + oblako_name, dtype='object', sheet_name='РЕЕСТР ')
oblako_spravochniki = pd.read_excel(oblako_path + oblako_name, dtype='object', sheet_name='Справочники')

In [3]:
oblako['Дата включения пациента в Реестр'] = pd.to_datetime(oblako['Дата включения пациента в Реестр'], dayfirst=True).dt.normalize()
oblako['Дата рождения'] = pd.to_datetime(oblako['Дата рождения'], dayfirst=True, errors ='coerce').dt.normalize()
oblako['Дата прикрепления к патронажному участку'] = pd.to_datetime(oblako['Дата прикрепления к патронажному участку'], dayfirst=True, errors ='coerce').dt.normalize()
oblako['Дата выдачи заключения'] = pd.to_datetime(oblako['Дата выдачи заключения'], dayfirst=True, errors ='coerce').dt.normalize()
oblako['Дата выдачи разрешения'] = pd.to_datetime(oblako['Дата выдачи разрешения'], dayfirst=True, errors ='coerce').dt.normalize()
oblako['Плановая дата первичного визита 24 часа'] = pd.to_datetime(oblako['Плановая дата первичного визита 24 часа'], dayfirst=True, errors ='coerce').dt.normalize()
oblako['Плановая дата первичного визита 72 часа'] = pd.to_datetime(oblako['Плановая дата первичного визита 72 часа'], dayfirst=True, errors ='coerce').dt.normalize()
oblako['Плановая дата госпитализации'] = pd.to_datetime(oblako['Плановая дата госпитализации'], dayfirst=True, errors ='coerce').dt.normalize()
oblako['Дата прекращения оказания ПМП'] = pd.to_datetime(oblako['Дата прекращения оказания ПМП'], dayfirst=True, errors ='coerce').dt.normalize()
oblako['Дата смерти'] = pd.to_datetime(oblako['Дата смерти'], dayfirst=True, errors ='coerce').dt.normalize()
oblako['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'] = pd.to_datetime(oblako['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'], dayfirst=True, errors ='coerce').dt.normalize()
oblako['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'] = pd.to_datetime(oblako['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'], dayfirst=True, errors ='coerce').dt.normalize()
oblako['Дата фактической госпитализации из  ЕМИАС'] = pd.to_datetime(oblako['Дата фактической госпитализации из  ЕМИАС'], dayfirst=True, errors ='coerce').dt.normalize()
oblako['Последний визит врача из протокола осмотра ЕМИАС'] = pd.to_datetime(oblako['Последний визит врача из протокола осмотра ЕМИАС'], dayfirst=True, errors ='coerce').dt.normalize()
oblako['Плановая дата следующего визита из протокола осмотра ЕМИАС'] = pd.to_datetime(oblako['Плановая дата следующего визита из протокола осмотра ЕМИАС'], dayfirst=True, errors ='coerce').dt.normalize()
oblako['Дата  дозвона'] = pd.to_datetime(oblako['Дата  дозвона'], dayfirst=True, errors ='coerce').dt.normalize()

In [4]:
df_obl = oblako.copy()
df_obl_sprav = oblako_spravochniki.copy()

In [5]:
shutil.copy2(oblako_path + oblako_name, 'z:/Выездные службы/' + oblako_name)

'z:/Выездные службы/22-10-2024_16-00_Реестр ЦПП.xlsx'

In [6]:
os.chmod('z:/Выездные службы/' + oblako_name, stat.S_IREAD)

In [7]:
df_obl = df_obl.loc[(df_obl['Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях'] != 'Заполняется КЦ')
                    & ~(df_obl['Причина прекращения оказания ПМП'].str.contains('смерть', case=False, na=False))
                    ]

In [8]:
df_obl['ID пациента'] = df_obl['ID пациента'].astype('str')

In [9]:
list_filial = df_obl['Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях'].unique()

In [10]:
# Ячейки НЕ для корректировки

not_corrected = ['A1:AB1','AD1:AL1', 'AS1:AX1']

In [11]:
# Ячейки для корректировки

for_corrected = ['AC1:AC5000','AM1:AR5000','AY1:AY5000']

In [12]:
# Ячейки для визуального выделения

for_filling = ['B2:C5000','E2:E5000','L2:L5000']

In [13]:
path_save = 'z:/Выездные службы'

# path_save = 'c:/Users/1/Desktop/выгрузки/test/'

In [14]:
for i in tqdm(list_filial, desc='Сохранение файлов', unit='файл'):
    file_name =  f'{path_save}/{i}/{datetime.now().strftime("%Y%m%d")}_{i}.xlsx'
    file_name = file_name.replace('"', '')
    df_save = df_obl.loc[df_obl['Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях'] == i]
    with pd.ExcelWriter(file_name,
                        engine="xlsxwriter",
                        date_format='DD.MM.YYYY',
                        datetime_format='DD.MM.YYYY'
                       ) as writer:        
        
        df_save.to_excel(writer, freeze_panes=(1,0), index = False)
        worksheet = writer.sheets["Sheet1"]
               
        format1 = writer.book.add_format({'bg_color': '#dddddd'})
        format2 = writer.book.add_format({'bg_color': '#99ff99'})
        format3 = writer.book.add_format({'bg_color': '#99ccff'})
        
        unlocked = writer.book.add_format({"locked": False})
        locked = writer.book.add_format({"locked": True})
        
        # Устанавливаем защиту листа и пароль
        # worksheet.protect('paRol7!')
        
        for i in not_corrected:
            worksheet.conditional_format(i, # Заливка серым ячеек не для корректирования
                                         {'type' : 'cell',
                                        'criteria' : '<>',
                                        'value' : 0,
                                        'format' : format1})
        
        for i in for_corrected:
            worksheet.conditional_format(i, # Заливка ячеек для корректирования
                                         {'type' : 'text',
                                        'criteria' : 'not containing',
                                        'value' : 'abrakadabra',
                                        'format' : format2})

        for i in for_filling:
            worksheet.conditional_format(i, # Заливка ячеек для визуального выделения
                                         {'type' : 'text',
                                        'criteria' : 'not containing',
                                        'value' : 'abrakadabra',
                                        'format' : format3})
        
        worksheet.set_column('A:AY', 2)
        worksheet.set_column('A:A', 10)
        worksheet.set_column('B:B', 30)
        worksheet.set_column('C:C', 15)
        worksheet.set_column('E:E', 10)
        worksheet.set_column('L:L', 18)
        worksheet.set_column('M:M', 7)
        worksheet.set_column('N:N', 18)
        worksheet.set_column('O:O', 20)
        worksheet.set_column('P:P', 12)
        worksheet.set_column('AD:AD', 6)
        worksheet.set_column('AE:AE', 10)
        worksheet.set_column('AJ:AL', 10)
        worksheet.set_column('AS:AX', 10)
        worksheet.set_column('AC:AC', 12, unlocked)
        worksheet.set_column('AM:AR', 12, unlocked)
        worksheet.set_column('AY:AY', 12, unlocked)
        worksheet.set_row(0, 100, locked)        
        
        worksheet.data_validation('AC2:AC5000', # Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях
                                  {'validate': 'list', 
                                  'source': '=sprav!$M$2:$M$20'})
        worksheet.data_validation('AM2:AM5000', # Текущий статус пациента
                                  {'validate': 'list', 
                                  'source': '=sprav!$S$2:$S$5'})
        worksheet.data_validation('AN2:AN5000', # Причина прекращения оказания ПМП
                                  {'validate': 'list', 
                                  'source': '=sprav!$T$2:$T$6'})
        worksheet.data_validation('AO2:AO5000', # Причины невыполнения «задания по пациенту»
                                  {'validate': 'list', 
                                  'source': '=sprav!$U$2:$U$5'})
        worksheet.data_validation('AP2:AP5000', # Причина прекращения на сопровождении
                                  {'validate': 'list', 
                                  'source': '=sprav!$U$2:$U$5'})
        worksheet.data_validation('AQ2:AQ5000', # Дата прекращения оказания ПМП
                                  {'validate': 'date',
                                  'criteria': 'between',
                                  'minimum': date(2000, 1, 1),
                                  'maximum': date(2030, 12, 31)})
        worksheet.data_validation('AR2:AR5000', # Дата смерти
                                  {'validate': 'date',
                                  'criteria': 'between',
                                  'minimum': date(2000, 1, 1),
                                  'maximum': date(2030, 12, 31)})
        workbook = writer.book
        worksheet = writer.sheets["Sheet1"]
              
        df_obl_sprav.to_excel(writer, index = False, sheet_name='sprav')

Сохранение файлов:   0%|          | 0/21 [00:00<?, ?файл/s]

Сохранение файлов: 100%|██████████| 21/21 [01:03<00:00,  3.03s/файл]


In [15]:
for i in tqdm(list_filial, desc='Изменение файлов', unit='файл'):
    file_name =  f'{path_save}/{i}/{datetime.now().strftime("%Y%m%d")}_{i}.xlsx'
    file_name = file_name.replace('"', '')
    workbook = load_workbook(filename=file_name)
    sheet = workbook.active
    sheet.auto_filter.ref = sheet.dimensions
    header_row = sheet[1]        
    for cell in header_row:
        cell.alignment = Alignment(wrap_text=True)        
    workbook.save(filename=file_name)

Изменение файлов: 100%|██████████| 21/21 [00:41<00:00,  1.98s/файл]
